Gradient accumulation

In [2]:
import torch
import sklearn

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits

In [3]:
x,y=sklearn.datasets.load_digits(return_X_y=True)
x=torch.tensor(x/16).float().cuda() # FP32
y=torch.tensor(y).long().cuda()
print(x.shape,x.dtype)
print(y.shape,y.dtype)

torch.Size([1797, 64]) torch.float32
torch.Size([1797]) torch.int64


In [4]:
class MLP(torch.nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MLP,self).__init__()
        self.fc1=torch.nn.Linear(input_size, hidden_size)
        self.fc2=torch.nn.Linear(hidden_size, output_size)

    def forward(self,x):
        out=self.fc1(x)
        out=torch.relu(out)
        out=self.fc2(out)
        return out

In [5]:
model=MLP(input_size=64,hidden_size=256,output_size=10).cuda()
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

iter=0
accum_steps=0
while True:
    out=model(x)
    loss=loss_fn(out,y)
    loss=loss/4    
    loss.backward()
    accum_steps+=1
    if accum_steps==4:
        optimizer.step()
        optimizer.zero_grad()
        accum_steps=0
        iter+=1
        if iter%25000==0:
            print(f'iter={iter} loss={loss.item()} cuda_mem={torch.cuda.memory_allocated()}Bytes')
        if loss.item()<=1e-3:
            break

iter=25000 loss=0.011231090873479843 cuda_mem=17916416Bytes
iter=50000 loss=0.005173448007553816 cuda_mem=17916416Bytes
iter=75000 loss=0.0030425109434872866 cuda_mem=17916416Bytes
iter=100000 loss=0.0020461517851799726 cuda_mem=17916416Bytes
iter=125000 loss=0.001496035954914987 cuda_mem=17916416Bytes
iter=150000 loss=0.0011577062541618943 cuda_mem=17916416Bytes
